Create annotations for xray images in appropriate Yolov3 format. 

Yolov3 format: 
Row format: image_file_path box1 box2 ... boxN;

path/to/img1.jpg 50,100,150,200,0 30,50,200,120,3

path/to/img2.jpg 120,300,250,600,2
...

In [1]:
import argparse
import io
import os
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
train_path='input\stage_1_train_images'
val_path='input\stage_1_test_images'
train_path = os.path.expanduser(train_path)
val_path = os.path.expanduser(val_path)
files = os.listdir(train_path)

In [34]:
train_labels = pd.read_csv('input/stage_1_train_labels.csv')
train_labels.head()

,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1


In [37]:
#https://www.kaggle.com/peterchang77/exploratory-data-analysis

def parse_data(df):
    """
    Method to read a CSV file (Pandas dataframe) and parse the 
    data into the following nested dictionary:

      parsed = {
        
        'patientId-00': {
            'dicom': path/to/dicom/file,
            'label': either 0 or 1 for normal or pnuemonia, 
            'boxes': list of box(es)
        },
        'patientId-01': {
            'dicom': path/to/dicom/file,
            'label': either 0 or 1 for normal or pnuemonia, 
            'boxes': list of box(es)
        }, ...

      }

    """
    # --- Define lambda to extract coords in list [y, x, height, width]
    extract_box = lambda row: [row['y'], row['x'], row['height'], row['width']]

    parsed = {}
    for n, row in df.iterrows():
        # --- Initialize patient entry into parsed 
        pid = row['patientId']
        if pid not in parsed:
            parsed[pid] = {
                'dicom': 'input/stage_1_train_images/%s.dcm' % pid,
                'label': row['Target'],
                'boxes': []}

        # --- Add box if opacity is present
        if parsed[pid]['label'] == 1:
            parsed[pid]['boxes'].append(extract_box(row))

    return parsed

In [38]:
parsed = parse_data(train_labels)

                              patientId      x      y  width  height  Target
0  0004cfab-14fd-4e49-80ba-63a80b6bddd6    NaN    NaN    NaN     NaN       0
1  00313ee0-9eaa-42f4-b0ab-c148ed3241cd    NaN    NaN    NaN     NaN       0
2  00322d4d-1c29-4943-afc9-b6754be640eb    NaN    NaN    NaN     NaN       0
3  003d8fa0-6bf1-40ed-b54c-ac657f8495c5    NaN    NaN    NaN     NaN       0
4  00436515-870c-4b36-a041-de91049b9ab4  264.0  152.0  213.0   379.0       1


In [39]:
train_labels = pd.read_csv('input/stage_1_train_labels.csv', index_col='patientId')
train_labels.head()

,x,y,width,height,Target
patientId,,,,,
0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1


In [40]:
N = len(train_labels.index.unique())
print(N)

25684


In [41]:
output_file=open('train1.txt','w')
for i in range(N):
    # component 1: data_path + image_name
    part1 = os.path.join(train_path,train_labels.index.unique()[i]+'.dcm')
    # component 2: box data. 
    data = parsed[train_labels.index.unique()[i]]
    boxes = data['boxes']
    part2 = ''
    if len(boxes)>0:
        for j in range(len(boxes)):
            bj = np.asarray(boxes[j])
            bjj=np.zeros(bj.shape)
            bjj[0]=bj[1]
            bjj[1]=bj[0]
            bjj[2]=bj[1]+bj[3]
            bjj[3]=bj[0]+bj[2]
            sj=np.array2string(bjj.astype(int),separator=',')
            sj1=sj.replace(" ", "")
            part2 = part2 + sj1[1:-1] + ',0 '
        fullline = part1 + ' ' + part2[:-1]
        output_file.write(fullline)
        output_file.write('\n')
        print(fullline)
#     else:
#         fullline = part1
#     output_file.write(fullline)
#     output_file.write('\n')
#     print(fullline)
output_file.close()    

input\stage_1_train_images\00436515-870c-4b36-a041-de91049b9ab4.dcm 264,152,477,531,0 562,152,818,605,0
input\stage_1_train_images\00704310-78a8-4b38-8475-49f4573b2dbb.dcm 323,577,483,681,0 695,575,857,712,0
input\stage_1_train_images\00aecb01-a116-45a2-956c-08d2fa55433f.dcm 288,322,382,457,0 547,299,666,464,0
input\stage_1_train_images\00c0b293-48e7-4e16-ac76-9269ba535a62.dcm 306,544,474,788,0 650,511,856,795,0
input\stage_1_train_images\00f08de1-517e-4652-a04f-d1dc9ee48593.dcm 181,184,387,690,0 571,275,801,751,0
input\stage_1_train_images\0100515c-5204-4f31-98e0-f35e4b00004a.dcm 703,416,787,493,0
input\stage_1_train_images\010ccb9f-6d46-4380-af11-84f87397a1b8.dcm 652,437,813,730,0 301,405,442,684,0
input\stage_1_train_images\012a5620-d082-4bb8-9b3b-e72d8938000c.dcm 133,613,408,888,0 678,427,902,767,0
input\stage_1_train_images\0174c4bb-28f5-41e3-a13f-a396badc18bd.dcm 155,182,428,683,0 599,220,826,728,0
input\stage_1_train_images\019d950b-dd38-4cf3-a686-527a75728be6.dcm 229,318,479,61

input\stage_1_train_images\06a4f477-34ca-4625-945b-81bfa9e77273.dcm 512,355,825,974,0
input\stage_1_train_images\06ac3c8c-cd1f-4e10-ac24-4cb4f3ab1243.dcm 503,271,865,938,0
input\stage_1_train_images\06b52ffd-71b8-429a-a284-bb39240ed343.dcm 496,116,875,739,0 102,129,409,631,0
input\stage_1_train_images\06b64e36-9e61-4865-bfe4-d96eef86548c.dcm 504,265,766,645,0
input\stage_1_train_images\06bc1519-c307-4a98-b00b-8329f2ddf615.dcm 217,325,511,581,0 592,253,904,681,0
input\stage_1_train_images\06bf9151-5732-4968-b198-f4109676cd55.dcm 548,233,925,882,0
input\stage_1_train_images\06c3e09b-341b-4fb9-9e28-bfd9cf298c6d.dcm 253,450,494,683,0
input\stage_1_train_images\06c50ad5-8ffd-480b-8424-dd38df7ca86c.dcm 574,99,899,843,0
input\stage_1_train_images\06cc46b7-50b1-41c1-a406-ccac7a5ac3e2.dcm 519,295,752,646,0 220,279,453,771,0
input\stage_1_train_images\06cc9b59-b8ff-4a6b-9e5b-f127c7c1208d.dcm 154,495,417,748,0
input\stage_1_train_images\06cf339f-a71a-4177-b950-ae1fdd5f4762.dcm 63,476,370,648,0 54

input\stage_1_train_images\0918aae8-b873-4510-b975-c6e87df90f58.dcm 461,185,790,936,0 127,452,415,981,0
input\stage_1_train_images\091cc2b7-8ba6-4fce-8ae2-7547117dbddf.dcm 131,200,386,855,0 559,239,856,974,0
input\stage_1_train_images\091d75a8-e23e-4cb8-af67-10cb314077ec.dcm 118,501,346,635,0 579,501,859,897,0
input\stage_1_train_images\091fa211-36c0-42a4-bd7d-c8bced664d9d.dcm 208,507,423,788,0
input\stage_1_train_images\0927e819-9640-487d-a04e-f5d4732dbe0a.dcm 536,336,762,598,0
input\stage_1_train_images\0930b0aa-25c2-4624-a3fc-9b60ba0b23f4.dcm 471,8,916,576,0 93,10,439,601,0
input\stage_1_train_images\09326eb7-f4cb-4d8f-83c6-8ba7fb8b5ac7.dcm 210,53,492,370,0
input\stage_1_train_images\0949d712-9d3c-424a-99b5-423a598d8b97.dcm 654,465,786,602,0
input\stage_1_train_images\094eed38-9c5b-4042-936d-344ccec4c3cc.dcm 122,350,422,800,0 544,346,841,866,0
input\stage_1_train_images\0953df2c-f969-4fa1-a872-022475ecd5ef.dcm 168,325,400,618,0 509,310,704,611,0
input\stage_1_train_images\095d6b7c-f

input\stage_1_train_images\0c0da1cf-9514-4c21-9988-fb53c52528d1.dcm 119,62,407,740,0 503,38,852,753,0
input\stage_1_train_images\0c0dbae5-b0a7-43b7-a964-11e6bcf40ae9.dcm 589,367,784,606,0 248,287,448,584,0
input\stage_1_train_images\0c122fd9-6dc8-4224-baae-812ed5c4bd12.dcm 232,106,439,346,0
input\stage_1_train_images\0c153d93-d65b-4d3e-90bc-8b26350e3476.dcm 106,504,349,763,0 566,398,767,770,0
input\stage_1_train_images\0c15e437-7eea-4ed1-b1d8-124f79cc1893.dcm 262,186,449,581,0
input\stage_1_train_images\0c1d54d9-0197-40d9-9b07-d99b1e8aa8db.dcm 356,557,563,791,0
input\stage_1_train_images\0c248828-72e2-43cf-8622-b5775c421fb1.dcm 551,207,759,480,0 193,418,410,570,0
input\stage_1_train_images\0c24e2a2-9fca-463a-a1fa-fbcda2712260.dcm 639,526,823,744,0
input\stage_1_train_images\0c294ecf-23d6-4c56-b5e2-0482915cd102.dcm 544,145,864,792,0 188,130,462,738,0
input\stage_1_train_images\0c2e5286-bb2c-427f-98be-8fc3796383d5.dcm 240,495,427,712,0 600,643,913,876,0
input\stage_1_train_images\0c391e0

input\stage_1_train_images\13ef2d8e-3b2e-4fbb-9f70-69139dd74401.dcm 192,358,361,692,0 558,327,739,717,0
input\stage_1_train_images\1414c7d6-dff3-453c-8322-b25bb5899d78.dcm 114,311,265,643,0 635,319,790,737,0
input\stage_1_train_images\142c7419-9cd2-4436-8860-cd2c4355abff.dcm 582,385,853,829,0 123,432,379,736,0
input\stage_1_train_images\14996682-7fb0-42e1-a925-ab2a639bae18.dcm 151,500,356,681,0
input\stage_1_train_images\149d826e-7078-4b2c-888a-d8811c72caf8.dcm 661,430,821,627,0 221,428,400,630,0
input\stage_1_train_images\14a7fbc6-6661-4382-882f-b2aa317cadc0.dcm 705,380,864,591,0 210,329,379,525,0
input\stage_1_train_images\152f5cd9-cc37-4312-befb-1b036cb68c82.dcm 318,178,483,394,0
input\stage_1_train_images\153dc47f-9a10-4cbe-a0eb-8a747cc6d8b8.dcm 292,229,494,602,0 634,223,850,686,0
input\stage_1_train_images\15508c0e-a6bd-4d3a-9be2-76466b9f7eb1.dcm 560,488,848,765,0 228,407,476,739,0
input\stage_1_train_images\1551a600-9f33-4837-b932-7514feac3789.dcm 122,457,378,739,0
input\stage_1_

...

input\stage_1_train_images\e9e58cf9-f301-4fe9-a5d5-ff12f7981612.dcm 203,343,373,651,0 640,506,754,626,0
input\stage_1_train_images\e9e58e10-2795-4dec-a738-c9792ea8f685.dcm 144,366,391,814,0 582,412,810,784,0
input\stage_1_train_images\e9ea9123-3eab-4d60-973d-bfca20b6e054.dcm 337,387,552,861,0
input\stage_1_train_images\ea1cf22d-0ec1-4820-86ed-a2bce81fa924.dcm 541,486,719,609,0
input\stage_1_train_images\ea20f7ad-5958-4a6a-b871-89e747593fd8.dcm 244,477,424,572,0
input\stage_1_train_images\ea2f0e9e-8cca-4037-a456-9fb1f07af5d2.dcm 202,503,371,655,0
input\stage_1_train_images\ea3d1dcb-9222-4e66-b7e6-a45e78bd339e.dcm 330,292,481,516,0
input\stage_1_train_images\ea3f3fb8-d6dc-41c8-9bdf-a67b43570477.dcm 311,559,456,669,0
input\stage_1_train_images\ea3f8b01-bbf4-4abe-ba79-72b6989f005b.dcm 185,423,366,557,0
input\stage_1_train_images\ea475b41-c5ae-4d80-b5bf-d5af6f1b0c70.dcm 228,576,408,749,0
input\stage_1_train_images\ea4e3c77-6956-4ba1-9c73-3bc93c1f0398.dcm 108,699,315,787,0
input\stage_1_trai

input\stage_1_train_images\ee52c757-3e8e-4e65-9818-52ad4c00622a.dcm 297,217,479,513,0
input\stage_1_train_images\ee5842e0-5bd3-4e4b-9aea-cb94b6402fdb.dcm 143,419,367,756,0 636,552,808,752,0
input\stage_1_train_images\ee674127-083a-4562-a4e6-3ee96bae758a.dcm 272,463,399,630,0
input\stage_1_train_images\ee820aa5-4804-4984-97b3-f0a71d69702f.dcm 605,228,759,451,0 634,569,827,744,0 211,238,378,473,0 168,611,326,728,0
input\stage_1_train_images\ee8a4128-a907-475b-a363-9cd60d65e46e.dcm 644,434,826,781,0 212,365,396,743,0
input\stage_1_train_images\eeb9b9e9-1502-49c0-b622-94f43a0b07c8.dcm 598,554,754,684,0 616,378,717,514,0
input\stage_1_train_images\eec6148f-5a8e-48e0-967f-3f59f14c562f.dcm 552,253,747,638,0
input\stage_1_train_images\eee3497c-fb69-4a48-96e6-c204b0ee9b84.dcm 193,183,420,432,0
input\stage_1_train_images\eef875e3-f6ce-457d-bb4f-7c42bf3cce09.dcm 250,316,399,544,0
input\stage_1_train_images\ef0fe9c2-fb59-4631-86ec-c47020ff231b.dcm 553,314,759,853,0 201,373,409,832,0
input\stage_1_

input\stage_1_train_images\f32e03a8-4c73-49dc-a142-c6d8d55dff94.dcm 699,241,850,411,0
input\stage_1_train_images\f336113a-6002-41da-8451-997d62e36a34.dcm 277,241,430,526,0
input\stage_1_train_images\f34899e1-3927-4191-b50c-bec95808139c.dcm 604,356,753,555,0
input\stage_1_train_images\f37a0400-beea-4925-8b4b-420f1b33284b.dcm 150,357,291,546,0
input\stage_1_train_images\f3a72d62-fce7-4654-92a1-9cf26353d156.dcm 207,520,441,712,0
input\stage_1_train_images\f3aa1daa-4873-446a-a5b7-7da7f955245e.dcm 183,478,370,630,0 617,262,797,611,0
input\stage_1_train_images\f3b1a1b8-b6d9-4ac7-85ea-41765f99eaf8.dcm 466,184,661,415,0
input\stage_1_train_images\f3b1b61e-693d-4fee-b67b-fa4f09554391.dcm 561,394,692,549,0
input\stage_1_train_images\f3bea900-7094-44d2-bb12-fdec80766f9d.dcm 299,324,484,521,0
input\stage_1_train_images\f3d51616-15d7-419c-acae-90413b7fdcae.dcm 535,351,783,788,0 121,359,352,745,0
input\stage_1_train_images\f3d8b643-0706-4997-8819-92bb95524626.dcm 98,576,387,836,0
input\stage_1_train

input\stage_1_train_images\f7f3e746-b2d6-47e8-87fd-e7d9db9b1836.dcm 139,506,437,733,0
input\stage_1_train_images\f8153745-24f3-43d5-8684-209b59fcddfe.dcm 231,568,395,690,0
input\stage_1_train_images\f8377b25-0c03-4615-b18d-f0131b80b3d4.dcm 701,409,847,603,0 212,553,400,739,0
input\stage_1_train_images\f854d6cf-8297-4894-80fe-90f9a11d0cb0.dcm 600,551,768,790,0
input\stage_1_train_images\f86a02d1-4ee2-4578-b001-a03952254643.dcm 666,515,812,779,0 175,417,414,729,0
input\stage_1_train_images\f87332a3-9e9b-4405-bcb2-d52652adbaba.dcm 637,570,847,722,0
input\stage_1_train_images\f8767c5d-9024-43d9-9044-843f2cec00c7.dcm 669,342,871,856,0 189,353,447,744,0
input\stage_1_train_images\f87e319e-34e7-4d47-893f-8a004ffcb94f.dcm 125,258,386,623,0
input\stage_1_train_images\f8850ac3-bde9-473c-aca8-095f16193071.dcm 568,512,774,647,0
input\stage_1_train_images\f88e605a-a637-4df6-a153-5c51bb304a73.dcm 214,312,435,534,0 645,237,791,493,0
input\stage_1_train_images\f89cdd03-5ea2-4e80-98fb-1d2e57686638.dcm 

input\stage_1_train_images\fd45f813-2237-4639-b340-2e1b97737da6.dcm 726,454,865,708,0 268,438,459,709,0
input\stage_1_train_images\fd5cc454-f85b-464d-a04f-09d46a86d200.dcm 325,277,535,612,0 770,366,971,645,0
input\stage_1_train_images\fd676e0b-14e6-40d4-8c4a-85d028559ad1.dcm 632,284,741,485,0 200,185,366,381,0
input\stage_1_train_images\fd67d636-d57c-4698-876b-23e94eed5067.dcm 178,251,416,556,0
input\stage_1_train_images\fd72c1b0-9525-4bcc-b70e-15e1755c1b7a.dcm 175,425,317,846,0
input\stage_1_train_images\fd8de63f-e006-4e20-a937-9ecbde6e31e9.dcm 215,235,460,743,0 597,256,852,761,0
input\stage_1_train_images\fdc8b07c-8c97-4022-ac84-347f8072f07a.dcm 635,389,829,707,0
input\stage_1_train_images\fdd20e4d-879e-4ff9-bc4e-8ec4184cfff9.dcm 190,460,368,643,0 658,484,838,795,0
input\stage_1_train_images\fde7bf44-9cb7-4506-9738-5774d4547f6f.dcm 288,478,412,634,0
input\stage_1_train_images\fde83ad1-3d9e-4a66-8f10-bb6ead510d52.dcm 270,213,458,520,0 638,239,812,574,0
input\stage_1_train_images\fde9b

In [42]:
parsed

{'796c0726-bde4-4ff2-aa00-11587b3a8a0c': {'boxes': [],
  'dicom': 'input/stage_1_train_images/796c0726-bde4-4ff2-aa00-11587b3a8a0c.dcm',
  'label': 0},
 '3c06574a-0c86-4a9d-bd6e-ad4e380b8082': {'boxes': [],
  'dicom': 'input/stage_1_train_images/3c06574a-0c86-4a9d-bd6e-ad4e380b8082.dcm',
  'label': 0},
 '6a87f637-edc6-4ba4-99a3-65ad4de67f71': {'boxes': [],
  'dicom': 'input/stage_1_train_images/6a87f637-edc6-4ba4-99a3-65ad4de67f71.dcm',
  'label': 0},
 '4a2c6a7f-6a4d-4c3a-ab8b-ad83795f136a': {'boxes': [],
  'dicom': 'input/stage_1_train_images/4a2c6a7f-6a4d-4c3a-ab8b-ad83795f136a.dcm',
  'label': 0},
 '8fedf466-d1bb-4487-8642-c18f8c428bd3': {'boxes': [],
  'dicom': 'input/stage_1_train_images/8fedf466-d1bb-4487-8642-c18f8c428bd3.dcm',
  'label': 0},
 '09009286-a589-4fb9-8c2c-540e16c75481': {'boxes': [],
  'dicom': 'input/stage_1_train_images/09009286-a589-4fb9-8c2c-540e16c75481.dcm',
  'label': 0},
 '7184cc40-a58a-40df-9aa7-39d610190410': {'boxes': [],
  'dicom': 'input/stage_1_train_i